In [2]:
import requests
import pandas as pd
import numpy as np
import requests
import bs4
import ssl
import certifi
import time

In [3]:
def clean_rules(file):
    rules = pd.read_excel(file).T
    rules = rules.rename(columns=rules.iloc[0]).drop(rules.index[0])
    rules = rules[rules.index.str.contains('Result')]
    rules.replace('Not Evaluated', 'No', inplace = True)
    rules.replace('Passed', 'Yes', inplace = True)
    rules.index = rules.index.str.replace(';Result', '')
    return rules  

In [4]:
def clean_data(data):
    final = clean_rules(data)
    final.replace('No', 0, inplace = True)
    final.replace('Yes', 1, inplace = True)
    final = final.T
    conditions = [(final['Google Universal Exists'] == 1) | (final['GA4 Exists'] == 1), (final['Google Universal Exists'] == 0) & (final['GA4 Exists'] == 0)]
    values = [1, 0]
    final['Google Analytics Exists'] = np.select(conditions, values)
    final['Adobe and Google Analytics'] = final['Adobe Ananlytics Exists'] + final['Google Analytics Exists']
    final['Adobe and Google Analytics'].replace(1, 0, inplace = True)
    final['Adobe and Google Analytics'].replace(2, 1, inplace = True)
    final['Adobe and Tealium TMS'] = final['Adobe Ananlytics Exists'] + final['Tealium TMS Exists']
    final['Adobe and GTM'] = final['Adobe Ananlytics Exists'] + final['GTM exists']
    final['Adobe and Ensighten'] = final['Adobe Ananlytics Exists'] + final['Ensighten Exists']
    final['Adobe and Adobe DTM'] = final['Adobe Ananlytics Exists'] + final['Adobe DTM Exists']
    final['Adobe and Adobe Target'] = final['Adobe Ananlytics Exists'] + final['Adobe Target Exists']
    final['Google and Tealium TMS'] = final['Google Analytics Exists'] + final['Tealium TMS Exists']
    final['Google and GTM'] = final['Google Analytics Exists'] + final['GTM exists']
    final['Google and Ensighten'] = final['Google Analytics Exists'] + final['Ensighten Exists']
    final['Google and Adobe DTM'] = final['Google Analytics Exists'] + final['Adobe DTM Exists']
    final['Google and Google Optimize'] = final['Google Analytics Exists'] + final['Google Optimize Exists']
    final[['Adobe and Tealium TMS', 'Adobe and GTM', 'Adobe and Ensighten', 'Adobe and Adobe DTM', 'Google and Tealium TMS', 'Google and GTM', 'Google and Ensighten', 'Google and Adobe DTM', 'Adobe and Adobe Target', 'Google and Google Optimize']] = final[['Adobe and Tealium TMS', 'Adobe and GTM', 'Adobe and Ensighten', 'Adobe and Adobe DTM', 'Google and Tealium TMS', 'Google and GTM', 'Google and Ensighten', 'Google and Adobe DTM', 'Adobe and Adobe Target', 'Google and Google Optimize']].replace(1, 0)
    final[['Adobe and Tealium TMS', 'Adobe and GTM', 'Adobe and Ensighten', 'Adobe and Adobe DTM', 'Google and Tealium TMS', 'Google and GTM', 'Google and Ensighten', 'Google and Adobe DTM', 'Adobe and Adobe Target', 'Google and Google Optimize']] = final[['Adobe and Tealium TMS', 'Adobe and GTM', 'Adobe and Ensighten', 'Adobe and Adobe DTM', 'Google and Tealium TMS', 'Google and GTM', 'Google and Ensighten', 'Google and Adobe DTM', 'Adobe and Adobe Target', 'Google and Google Optimize']].replace(2, 1)
    final['GA + Google Optimize + GTM'] = final['Google Analytics Exists'] + final['Google Optimize Exists'] + final['GTM exists']
    final['AA + Adobe Target + Adobe DTM'] = final['Adobe Ananlytics Exists'] + final['Adobe Target Exists'] + final['Adobe DTM Exists']
    final[['GA + Google Optimize + GTM', 'AA + Adobe Target + Adobe DTM']] = final[['GA + Google Optimize + GTM', 'AA + Adobe Target + Adobe DTM']].replace(1, 0)
    final[['GA + Google Optimize + GTM', 'AA + Adobe Target + Adobe DTM']] = final[['GA + Google Optimize + GTM', 'AA + Adobe Target + Adobe DTM']].replace(2, 0)
    final[['GA + Google Optimize + GTM', 'AA + Adobe Target + Adobe DTM']] = final[['GA + Google Optimize + GTM', 'AA + Adobe Target + Adobe DTM']].replace(3, 1)

    return final

In [5]:
final = clean_data('SDRAuditFinalAttempt3_Rules_2736279_85d03acfcc4420718348814caa619268.xlsx')

In [6]:
top_50 = final.iloc[:50]
next_200 = final.iloc[50:251]
next_249 = final.iloc[251:500]
next_499 = final.iloc[500:1000]

In [7]:
final

,GTM exists,Adobe Ananlytics Exists,Google Universal Exists,Adobe Launch Exists,Adobe DTM Exists,Tealium TMS Exists,Ensighten Exists,GA4 Exists,Mixpanel Exists,Heap Exists,...,Adobe and Ensighten,Adobe and Adobe DTM,Adobe and Adobe Target,Google and Tealium TMS,Google and GTM,Google and Ensighten,Google and Adobe DTM,Google and Google Optimize,GA + Google Optimize + GTM,AA + Adobe Target + Adobe DTM
https://3myms.eaglegl.com/default.asp,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
https://agt.fandom.com/wiki/CZN,1,0,1,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
https://alberto-pants-usa.com/,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
https://alexanderbaldwin.com/,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
https://allengelhard.com/,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
https://www.yum.com/,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
https://www.zalecorp.com,0,1,1,0,0,0,1,0,0,0,...,1,0,0,0,0,1,0,0,0,0
https://www.zimmer.com,0,1,1,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
https://wwwint.progress-energy.com/,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
total_tags = final.T.sum()
final.sum()

GTM exists                         466
Adobe Ananlytics Exists            227
Google Universal Exists            671
Adobe Launch Exists                161
Adobe DTM Exists                    21
Tealium TMS Exists                  76
Ensighten Exists                    24
GA4 Exists                         109
Mixpanel Exists                      3
Heap Exists                         14
Amplitude Exists                     2
Snowplow Exists                      9
Adobe Target Exists                136
Google Optimize Exists              84
Optimizely Exists                   27
Monetate Exists                     17
Visual Website Optimizer Exists     15
Adobe Audience Manager Exists      305
Oracle BlueKai                     136
Salesforce DMP                     112
Nielsen DMP Exists                  92
Lotame Crowd Control Exists         63
Lytics Exists                        0
Segment Exists                       0
Tealium CDP Exists                  76
TrustArc Exists          